# Wind speed invertion

This notebook example show how to invert wind speed from sigma0, using models from LUT or GMF, using [xsarsea.windspeed.invert_from_model](../basic_api.rst#xsarsea.windspeed.invert_from_model)

In [ ]:
import xsarsea
from xsarsea import windspeed
import xarray as xr
import numpy as np
import holoviews as hv
hv.extension('bokeh')

In [ ]:
# optional debug messages
import logging
logging.basicConfig()
logging.getLogger('xsarsea.windspeed').setLevel(logging.INFO) # or .setLevel(logging.DEBUG)

## Read sarwing owi file

download more from https://cyclobs.ifremer.fr/static/sarwing_datarmor/processings/c39e79a/default/reports/shoc_dailyupdate_names/report.html (in the 'safe' column, download files named like `*-owi-xx-*.nc`)

In [ ]:
sarwing_owi_file = xsarsea.get_test_file('s1a-iw-owi-xx-20210909t130650-20210909t130715-039605-04AE83.nc')
sarwing_ds = xsarsea.read_sarwing_owi(sarwing_owi_file)
sarwing_ds



## Get ancillary wind

Ecmwf wind is stored in owi file in *geographical* (deg N/S) convention. `xsarsea.windspeed` need it relative to *xtrack* (ie antenna), as a complex number.

We use [xsarsea.geo_dir_to_xtrack](../basic_api.rst#xsarsea.geo_dir_to_xtrack) to convert `sarwing_ds.owiEcmwfWindDirection` (deg) to radians, relative to xtrack, using `sarwing_ds.owiHeading`


In [ ]:
owi_ecmwf_wind = sarwing_ds.owiEcmwfWindSpeed * np.exp(1j* xsarsea.geo_dir_to_xtrack(sarwing_ds.owiEcmwfWindDirection, sarwing_ds.owiHeading))
sarwing_ds = xr.merge([
    sarwing_ds,
    owi_ecmwf_wind.to_dataset(name='owi_ancillary_wind'),
])

## Ancillary wind control plot

We check that ancillary wind is correct, by plotting the wind speed, the wind direction, and the wind direction components.

The *xtrack* component increase with *xtrack* axis, and *atrack* component increase with *atrack*.

Note that we have to use `kdims=['xtrack','atrack']` to properly display the vectorfield, because memory order `['xtrack','atrack']` is geographycally transposed 

In [ ]:
sub_sarwing_ds = sarwing_ds.isel(atrack=slice(None, None, 10), xtrack=slice(None, None, 10))

vectorfield = hv.VectorField(
    (
        sub_sarwing_ds.xtrack, sub_sarwing_ds.atrack,
        xsarsea.geo_dir_to_xtrack(sub_sarwing_ds.owiEcmwfWindDirection, sub_sarwing_ds.owiHeading),
        sub_sarwing_ds.owiEcmwfWindSpeed
    )
)

owi_ecmwf_wind_xtrack = np.real(owi_ecmwf_wind)
owi_ecmwf_wind_atrack = np.imag(owi_ecmwf_wind)
(
    hv.Image(sarwing_ds.owiEcmwfWindSpeed, kdims=['xtrack','atrack']).opts(title='speed and dir', clim=(0,50), cmap='jet') * vectorfield
    + hv.Image(owi_ecmwf_wind_xtrack, kdims=['xtrack','atrack']).opts(title='xtrack component',cmap='bwr') * vectorfield
    + hv.Image(owi_ecmwf_wind_atrack, kdims=['xtrack','atrack']).opts(title='atrack component',cmap='bwr') * vectorfield
)

## Copol inversion

Copol wind inversion, using model and ancillary wind

In [ ]:
windspeed_co = windspeed.invert_from_model(
        sarwing_ds.owiIncidenceAngle,
        sarwing_ds.owiNrcs,
        ancillary_wind = sarwing_ds.owi_ancillary_wind,
        model='gmf_cmod5n')


## differences with sarwing

In [ ]:
windspeed_diff = windspeed_co - sarwing_ds.owiWindSpeed_Tab_copol
(
    (
        hv.Image(windspeed_co, kdims=['xtrack','atrack']).opts(title='xsarsea') 
        + hv.Image(sarwing_ds.owiWindSpeed_Tab_copol, kdims=['xtrack','atrack']).opts(title='sarwing' )
    ).opts(hv.opts.Image(cmap='jet', clim=(0,50))) 
    + hv.Image(windspeed_diff, kdims=['xtrack','atrack']).opts(cmap='jet', clim=(-0.2,0.2), title='xsarsea-sarwing')
).opts(hv.opts.Image(colorbar=True, tools=['hover']))

## Crosspol inversion

Sarwing crosspol inversion is done with model `sarwing_lut_cmodms1ahw`, that is only available from lut.

So we use [xsarsea.windspeed.register_all_sarwing_luts](../basic_api.rst#xsarsea.windspeed.register_all_sarwing_luts) to register this lut.


In [ ]:
sarwing_luts_subset_path = xsarsea.get_test_file('sarwing_luts_subset')
windspeed.sarwing_luts.register_all_sarwing_luts(sarwing_luts_subset_path)

To match sarwing outputs, we will flatten `sarwing_ds.owiNesz_cross` with [xsarsea.windspeed.nesz_flattening](../basic_api.rst#xsarsea.windspeed.nesz_flattening) , and compute `dsig_cr`

In [ ]:
# nesz cross flattening, and dsig_cr
nesz_cross_flat = windspeed.nesz_flattening(sarwing_ds.owiNesz_cross, sarwing_ds.owiIncidenceAngle)
dsig_cr = (1.25 / (sarwing_ds.owiNrcs_cross / nesz_cross_flat )) ** 4.

windspeed_cr = windspeed.invert_from_model(
        sarwing_ds.owiIncidenceAngle,
        sarwing_ds.owiNrcs_cross,
        dsig_cr=dsig_cr,
        model='sarwing_lut_cmodms1ahw')

In [ ]:
windspeed_diff = windspeed_cr - sarwing_ds.owiWindSpeed_Tab_crosspol
(
    (hv.Image(windspeed_cr).opts(title='xsarsea') + hv.Image(sarwing_ds.owiWindSpeed_Tab_crosspol).opts(title='sarwing' )).opts(hv.opts.Image(cmap='jet', clim=(0,50))) +  
    hv.Image(windspeed_diff).opts(cmap='jet', clim=(-0.2,0.2), title='xsarsea-sarwing')
).opts(hv.opts.Image(colorbar=True, tools=['hover']))

## Dualpol inversion

Dualpol give better wind inversion results.

Dualpol inversion also returns copol wind.

In [ ]:
sarwing_luts_subset_path = xsarsea.get_test_file('sarwing_luts_subset')
windspeed.register_all_sarwing_luts(sarwing_luts_subset_path)

windspeed_co, windspeed_dual = windspeed.invert_from_model(
        sarwing_ds.owiIncidenceAngle,
        sarwing_ds.owiNrcs,
        sarwing_ds.owiNrcs_cross,
        ancillary_wind=sarwing_ds.owi_ancillary_wind,
        dsig_cr = dsig_cr,
        model=('gmf_cmod5n','sarwing_lut_cmodms1ahw'))

## differences with sarwing

In [ ]:
windspeed_diff = windspeed_dual - sarwing_ds.owiWindSpeed_Tab_dualpol_2steps
(
    (hv.Image(windspeed_dual).opts(title='xsarsea') + 
     hv.Image(sarwing_ds.owiWindSpeed_Tab_dualpol_2steps).opts(title='sarwing' )).opts(hv.opts.Image(cmap='jet', clim=(0,50))) +  
    hv.Image(windspeed_diff).opts(cmap='jet', clim=(-0.2,0.2), title='xsarsea-sarwing')
).opts(hv.opts.Image(colorbar=True, tools=['hover']))